# Proyecto Final - Grupo 1

## Integrantes

* Franco Gaito
* Francisco De Lorenzi
* Julieta Herrera
* Alejandro Relañez
* Cristian Barrera

## Instalaciones

Ejecutar la primera vez y luego comentar

In [15]:
 !pip install arch
 !pip install pmdarima

## Imports

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import itertools
import pmdarima as pm
import warnings

from typing import List, Dict, Any

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acovf
from statsmodels.tsa.stattools import adfuller, kpss

from arch.unitroot import ZivotAndrews

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Funciones

In [17]:
def dropear_columnas(df_x, columnas):
    """
    Realiza modificaciones en un DataFrame dado, incluyendo eliminación de columnas específicas y
    eliminación de filas con valores NaN en 'fake_positive'.
    """
    df = df_x.copy()
    df = df.drop(columns=columnas, axis=1)
    return df


def fix_types_and_nan(df: pd.DataFrame, config: Dict[str, Dict[str, Any]]) -> pd.DataFrame:
    """
    Función para preprocesar un dataframe según una configuración proporcionada.
    Rellena los valores NA y convierte los tipos de datos. Genera un error si se
    encuentran valores NA en una columna donde 'na_fill_value' está configurado como
    'NOT_ALLOWED'.

    Nota: Devuelve un dataframe que SOLO contiene las columnas especificadas.

    El parámetro config debe ser un diccionario que mapea nombres de columnas a
    otro diccionario que especifica el tipo de dato para la columna ('dtype') y
    cómo manejar los valores NA ('na_fill_value'). Si 'na_fill_value' está
    configurado como 'NOT_ALLOWED', la función generará un ValueError si se
    encuentran valores NA en esa columna.

    Ejemplo:

    config = {
        'promotion_id': {'dtype': str, 'na_fill_value': 'NOT_ALLOWED'},
        'days_of_week': {'dtype': str, 'na_fill_value': 'NOT_ALLOWED'},
        'cap_amount': {'dtype': np.float32, 'na_fill_value': 0},
        'slug': {'dtype': str, 'na_fill_value': 'NOT_ALLOWED'},
        'promotion_banks': {'dtype': str, 'na_fill_value': 'NOT_ALLOWED'},
        'pct_promo_value': {'dtype': np.float32, 'na_fill_value': 0},
        'visibility_start_date': {'dtype': 'datetime64[ns]', 'na_fill_value': 'NOT_ALLOWED'},
        'visibility_stop_date': {'dtype': 'datetime64[ns]', 'na_fill_value': 'NOT_ALLOWED'}
    }

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame de entrada a ser procesado.

    config : dict
        Diccionario que especifica los tipos de datos y el manejo de valores NA para cada columna.

    Returns
    -------
    df : pandas.DataFrame
        Dateframe procesado
    """
    df_copy = df.copy()
    for col, conf in config.items():
        if col in df_copy.columns:
            if conf['na_fill_value'] == "NOT_ALLOWED":
                if df_copy[col].isnull().any():
                    raise ValueError(f"NA valores encontrados en columna {col} que no admite valores NA.")
                df_copy[col] = df_copy[col].astype(conf['dtype'])
            else:
                df_copy[col] = df_copy[col].fillna(conf['na_fill_value']).astype(conf['dtype'])
        else:
            raise KeyError(f"{col} no encontrado en dataframe.")

    return df_copy[list(config.keys())]


def plot_acf_pacf(df, title='FAS y FACP de la Serie Diferenciada'):
    # Set up the plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(title)

    # Plot ACF and PACF
    plot_acf(df, ax=axes[0], title='FAS')
    plot_pacf(df, ax=axes[1], title='FACP')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


def plot_fac(df, title='FAC de la Serie Diferenciada'):
    # Set up the plot
    fig, ax = plt.subplots(figsize=(15, 5))
    fig.suptitle(title)

    # Plot FAC
    fac_values = acovf(df, adjusted=True)
    sns.lineplot(x=range(len(fac_values)), y=fac_values, ax=ax)
    ax.set_title('FAC')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


def stationarity_tests(df, title=''):
    print(f'Stationarity Tests for: {title}')

    # ADF Test
    print('ADF Test:')
    result_adf = adfuller(df, autolag='AIC')
    labels_adf = ['ADF Test Statistic', 'p-value', '# Lags Used', '# Observations']
    out_adf = pd.Series(result_adf[0:4], index=labels_adf)
    for key, val in result_adf[4].items():
        out_adf[f'Critical Value ({key})'] = val
    print(out_adf.to_string())
    print('\n')

    # KPSS Test
    print('KPSS Test:')
    result_kpss = kpss(df, regression='c', nlags='auto')
    labels_kpss = ['KPSS Test Statistic', 'p-value', '# Lags Used']
    out_kpss = pd.Series(result_kpss[0:3], index=labels_kpss)
    for key, val in result_kpss[3].items():
        out_kpss[f'Critical Value ({key})'] = val
    print(out_kpss.to_string())
    print('\n')


def test_stationarity(timeseries):
    print('Stationarity Tests:')

    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    # Graficar:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

    # Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    timeseries = timeseries.iloc[:,0].values
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)


def tes_optimizer(train, test, abg, trend_mode='add', seasonal_mode='add', seasonal_period=12, step=5):
    """
    This function optimizes hyperparameters for the TES model.
    """
    best_alpha, best_beta, best_gamma, best_mae = None, None, None, float("inf")
    for comb in abg:
        tes_model = ExponentialSmoothing(train, trend=trend_mode, seasonal=seasonal_mode, seasonal_periods=seasonal_period).\
            fit(smoothing_level=comb[0], smoothing_trend=comb[1], smoothing_seasonal=comb[2])
        y_pred = tes_model.forecast(step)
        mae = mean_absolute_error(test, y_pred)
        if mae < best_mae:
            best_alpha, best_beta, best_gamma, best_mae = comb[0], comb[1], comb[2], mae
        print([round(comb[0], 2), round(comb[1], 2), round(comb[2], 2), round(mae, 2)])
    print("best_alpha:", round(best_alpha, 2), "best_beta:", round(best_beta, 2), "best_gamma:", round(best_gamma, 2),
          "best_mae:", round(best_mae, 4))
    return best_alpha, best_beta, best_gamma, best_mae


def date_filter(df, column, inf, sup):
    return df[ (df[column].dt.year >= inf) & (df[column].dt.year <= sup) ]


def busqueda_sarimax(ts, valores_p, valores_d, valores_q, valores_P, valores_D, valores_Q, valores_s, exog=None):
    """
    Realiza una búsqueda en cuadrícula para encontrar los mejores parámetros del modelo SARIMAX.

    Parámetros:
    ts (pd.Series): Datos de la serie temporal.
    valores_p (list): Parte AR(p) del modelo.
    valores_d (list): Parte I(d) del modelo.
    valores_q (list): Parte MA(q) del modelo.
    valores_P (list): Parte AR(P) estacional del modelo.
    valores_D (list): Parte I(D) estacional del modelo.
    valores_Q (list): Parte MA(Q) estacional del modelo.
    valores_s (list): Períodos estacionales.
    exog (pd.DataFrame, opcional): Variables exógenas.

    Retorna:
    dict: Mejores parámetros y el AIC correspondiente.
    """
    warnings.filterwarnings("ignore")
    mejor_aic = float("inf")
    mejores_params = None

    pdq = list(itertools.product(valores_p, valores_d, valores_q))
    seasonal_pdq = list(itertools.product(valores_P, valores_D, valores_Q, valores_s))

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                model = SARIMAX(ts, order=param, seasonal_order=param_seasonal, exog=exog)
                results = model.fit(disp=False)
                if results.aic < mejor_aic:
                    mejor_aic = results.aic
                    mejores_params = (param, param_seasonal)
                print(f'SARIMAX{param}x{param_seasonal} - AIC:{results.aic}')
            except Exception as e:
                continue

    return {"params": mejores_params, "aic": mejor_aic}

## 1. Desarrollo

**Breve descripción del proyecto**

El proyecto se centra en el análisis de datos de aterrizajes y despegues de vuelos en Argentina, obtenidos del Ministerio de Transporte. El objetivo principal es comprender el comportamiento de los vuelos de Argentina a España para determinar la mejor fecha para viajar.

Para lograr esto, se realizará un análisis exploratorio de los datos, se aplicarán feature engineering y se construirán modelos predictivos utilizando algoritmos de machine learning. El resultado final será proporcionar recomendaciones sobre las fechas óptimas para viajar basadas en patrones identificados en los datos históricos de vuelos.






### 1.1) Importamos Dataset

In [18]:
# Leer los archivos CSV con el delimitador ';'
df_vuelos_argentina_2021 = pd.read_csv('../202112_informe_ministerio.csv', delimiter=',')
df_vuelos_argentina_2022 = pd.read_csv('../202212-informe-ministerio.csv', delimiter=';')
df_vuelos_argentina_2023 = pd.read_csv('../202312-informe-ministerio-actualizado-dic.csv', delimiter=';')
df_vuelos_argentina_2024 = pd.read_csv('../202404-informe-ministerio.csv', delimiter=';')

FileNotFoundError: [Errno 2] No such file or directory: '../202112_informe_ministerio.csv'

In [ ]:
# Renombrar la columna de fecha en df_vuelos_argentina_2021
df_vuelos_argentina_2021.columns.values[0] = df_vuelos_argentina_2022.columns[0]

# Combinar los DataFrames en uno solo
df_vuelos_combinado = pd.concat([df_vuelos_argentina_2021, df_vuelos_argentina_2022, df_vuelos_argentina_2023, df_vuelos_argentina_2024], ignore_index=True)

In [ ]:
# Mostrar las primeras filas del DataFrame combinado
df_vuelos_combinado.head()


### 1.2) Preprocessing

Renombramos columnas a snake_case

In [ ]:
df_vuelos_combinado.columns

In [ ]:
df_vuelos_combinado.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

In [ ]:
df_vuelos_combinado.rename(columns={'clase_de_vuelo_(todos_los_vuelos)': 'clase_de_vuelo'}, inplace=True)

In [ ]:
df_vuelos_combinado.rename(columns={'origen_/_destino': 'origen_destino'}, inplace=True)

In [ ]:
df_vuelos_combinado.columns

In [ ]:
df_vuelos_combinado.info()

In [ ]:
config = {
    'fecha_utc': {'dtype': 'datetime64[ns]', 'na_fill_value': 'NOT_ALLOWED'},
    'hora_utc': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'},
    'clase_de_vuelo': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'},
    'clasificación_vuelo': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'},
    'tipo_de_movimiento': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'},
    'aeropuerto': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'},
    'origen_destino': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'},
    'aerolinea_nombre': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'},
    'aeronave': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'},
    'pasajeros': {'dtype': 'int64', 'na_fill_value': 0},
    'pax': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'},
    'calidad_dato': {'dtype': 'string', 'na_fill_value': 'NOT_ALLOWED'}
}

In [ ]:
df_vuelos_combinado['fecha_utc'] = pd.to_datetime(df_vuelos_combinado['fecha_utc'], format='%d/%m/%Y', errors='coerce')

In [ ]:
df_vuelos_combinado['fecha_utc'].isna().sum()

In [ ]:
df_vuelos_combinado_correct_dtype = fix_types_and_nan(df_vuelos_combinado, config)

In [ ]:
df_vuelos_combinado_correct_dtype.info()

In [ ]:
# Extraer los años de la columna 'fecha'
años = df_vuelos_combinado_correct_dtype['fecha_utc'].dt.year

# Obtener los años únicos
años_unicos = años.unique()

# Mostrar los años únicos
print(años_unicos)

In [ ]:
df_vuelos_combinado_correct_dtype.shape

In [ ]:
df_vuelos_combinado_correct_dtype.isna().sum()

In [ ]:
df_vuelos_combinado_correct_dtype.describe()

### 1.2) EDA


Buscamos entender el comportamiento y distribución de los datos del dataset para entender si tenemos suficiente información y cómo podemos utilizarla.


---






Analisis de clases de vuelo

In [ ]:
# Obtener la cuenta de cada clase de vuelo
clase_vuelo_counts = df_vuelos_combinado_correct_dtype['clase_de_vuelo'].value_counts()

# Crear un gráfico de barras
plt.figure(figsize=(10, 5))
sns.barplot(x=clase_vuelo_counts.index, y=clase_vuelo_counts.values, hue=clase_vuelo_counts.index, legend=False, palette='viridis')
plt.title('Distribución de Clases de Vuelo en Argentina')
plt.xlabel('')
plt.ylabel('Cantidad de Vuelos')
plt.xticks(rotation=90)
plt.show()

Análisis de clasificaciónes de vuelo

In [ ]:
# Obtener la cuenta de cada clasificación de vuelo
clase_vuelo_counts = df_vuelos_combinado_correct_dtype['clasificación_vuelo'].value_counts()

# Calcular los porcentajes sobre el total
total_vuelos = len(df_vuelos_combinado_correct_dtype)
porcentajes = (clase_vuelo_counts / total_vuelos) * 100

# Crear una gráfica de barras
plt.figure(figsize=(10, 5))
barplot = sns.barplot(x=porcentajes.index, y=porcentajes.values, hue=porcentajes.index, legend=False, palette='viridis')

# Agregar etiquetas de porcentaje sobre las barras
for i in range(len(porcentajes)):
    plt.annotate(f'{porcentajes[i]:.2f}%',
                 xy=(i, porcentajes[i]),
                 ha='center',
                 va='bottom')

plt.title('Distribución de Clasificación de Vuelo en Argentina')
plt.xlabel('')
plt.xticks(rotation=45)
plt.gca().axes.get_yaxis().set_visible(False)  # Ocultar eje y
plt.show()



---



**Conclusiones:**

* La mayoría de los vuelos son domésticos



---






Top 50 de columna origen/destino

In [ ]:
# Obtener el top 50 de la columna 'origen_destino'
top_50_origen_destino = df_vuelos_combinado_correct_dtype['origen_destino'].value_counts().head(50)

# Mostrar los valores del top 5
top_50_origen_destino.reset_index()

**Conclusiones:**

Analizando el top 50 de vuelos origen/destino podemos ver que LEMD (Madrid) se encuentra en el puesto 39, sería conveniente utilizar este destino para realizar el pronóstico de viajes a España.

Nos quedamos solo con vuelos a españa (Todos los destinos para comparar)

In [ ]:
# Filtrar el DataFrame para incluir solo vuelos entre Argentina y España
aeropuertos=['LEMD', 'LEBL', 'MAD']
df_vuelos_argentina_españa_todos_los_destinos = df_vuelos_combinado_correct_dtype[df_vuelos_combinado_correct_dtype['origen_destino'].isin(aeropuertos)]

In [ ]:
# Contar la cantidad de vuelos por aeropuerto
cantidad_vuelos_por_aeropuerto = df_vuelos_argentina_españa_todos_los_destinos['origen_destino'].value_counts()

# Crear un gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(x=cantidad_vuelos_por_aeropuerto.index, y=cantidad_vuelos_por_aeropuerto.values, hue = cantidad_vuelos_por_aeropuerto.index, palette='viridis', legend=False)
plt.title('Cantidad de Vuelos por Aeropuerto (Argentina y España)')
plt.xlabel('Aeropuerto')
plt.ylabel('Cantidad de Vuelos')
for index, value in enumerate(cantidad_vuelos_por_aeropuerto):
    barplot.text(index, value, str(value), ha='center', va='bottom', fontsize=8)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Observamos un caso que nos llamó la atención de un vuelo a España desde Bahía Blanca. Observamos que se trata de un Jet privado
análisis_puntual='BCA'
df_vuelos_puntual = df_vuelos_combinado_correct_dtype[df_vuelos_combinado_correct_dtype["aeropuerto"] == análisis_puntual]
df_vuelos_puntual = df_vuelos_puntual[df_vuelos_puntual['origen_destino'].isin(aeropuertos)]
df_vuelos_puntual

Filtramos solo vuelos a LEMD para nuestro análisis

In [ ]:
# Filtramos el DataFrame para incluir solo vuelos entre Argentina y España ('LEMD')
aeropuerto_elegido ='LEMD'
df_vuelos_argentina_españa = df_vuelos_combinado_correct_dtype[df_vuelos_combinado_correct_dtype['origen_destino'].isin([aeropuerto_elegido])]

In [ ]:
df_vuelos_argentina_españa.head(5)

In [ ]:
df_vuelos_argentina_españa.shape

Analizamos cantidad de pasajeros por mes

In [ ]:
monthly_passengers = df_vuelos_argentina_españa.copy()
monthly_passengers.set_index('fecha_utc', inplace=True)
monthly_passengers = monthly_passengers['pasajeros'].resample('M').sum()
monthly_passengers.head(10)

In [ ]:
plt.figure(figsize=(15, 5))
monthly_passengers.plot()
plt.title('Número de pasajeros por mes')
plt.xlabel('Fecha')
plt.ylabel('Número de pasajeros')
plt.show()

Nuestro objetivo es extrapolar la demanda según fecha con los precios de vuelo para poder predecir cuando volar más barato. En ese caso, nos enfocaremos en un caso práctico que es vuelos que conecten Argentina y España. Por ese motivo, para el estudio nos quedaremos con:

*   Para el entrenamiento, vuelos entre 2021 y 2023. Para la prueba, vuelos de 2024
*   Vuelos que involucren el aeropuerto español LEMD
*   Vuelos comerciales
*   Contemplamos tanto despegues como aterrizajes, es decir, IDA y VUELTA
* Eliminamos filas con fecha o cantidad de pasajeros nulos

In [ ]:
aeropuerto_españa = ['LEMD']
clase_vuelo = 'Regular'

# Filtrar el DataFrame original
df_argentina_españa_comerciales = df_vuelos_combinado_correct_dtype[
    (df_vuelos_combinado_correct_dtype['origen_destino'].isin(aeropuerto_españa)) &
    (df_vuelos_combinado_correct_dtype['clase_de_vuelo'] == clase_vuelo)
]

num_vuelos = df_argentina_españa_comerciales.shape[0]
print(f"El número de vuelos en el dataset filtrado es de: {num_vuelos}")

In [ ]:
# Crear el dataset_train con registros entre 2021 y 2023
lim_inferior_entrenamiento=2021
lim_superior_entrenamiento=2023
lim_inferior_prueba=2024
lim_superior_prueba=2024

dataset_train = df_argentina_españa_comerciales[
    (df_argentina_españa_comerciales['fecha_utc'].dt.year >= lim_inferior_entrenamiento) &
    (df_argentina_españa_comerciales['fecha_utc'].dt.year <= lim_superior_entrenamiento)
]

# Crear el dataset_test con registros del año 2024
dataset_test = df_argentina_españa_comerciales[
    (df_argentina_españa_comerciales['fecha_utc'].dt.year >= lim_inferior_prueba) &
    (df_argentina_españa_comerciales['fecha_utc'].dt.year <= lim_superior_prueba)
]

# Imprimir el número de registros en cada dataset
print(f"El número de registros en dataset_train es: {len(dataset_train)}")
print(f"El número de registros en dataset_test es: {len(dataset_test)}")

Evaluamos gráficamente la cantidad de pasajeros por mes del DATASET de entrenamiento que contará con todos los mesese de los años 2021 a 2023, arrojandonos un resultado más concreto de como se desarrolla la variación en función del tiempo

In [ ]:
monthly_passengers_filtering = dataset_train.copy()
monthly_passengers_filtering.set_index('fecha_utc', inplace=True)
monthly_passengers_filtering = monthly_passengers_filtering['pasajeros'].resample('M').sum()
monthly_passengers_filtering.reset_index()
monthly_passengers_filtering.head(10)

In [ ]:
plt.figure(figsize=(15, 5))
monthly_passengers_filtering.plot()
plt.title('Número de pasajeros por mes')
plt.xlabel('Fecha')
plt.ylabel('Número de pasajeros')
plt.show()

NameError: name 'monthly_passengers_filtering' is not defined

<Figure size 1500x500 with 0 Axes>

Identificamos los aeropuertos argentinos, las aerolíneas y las aeronaves que guardan vínculo con estos viajes a España para vuelos comerciales en dicha fecha:

In [ ]:
# Gráfico de barra para la columna 'aeropuerto'
cantidad_vuelos_por_aeropuerto = df_argentina_españa_comerciales['aeropuerto'].value_counts()

plt.figure(figsize=(10, 5))
sns.barplot(x=cantidad_vuelos_por_aeropuerto.index, y=cantidad_vuelos_por_aeropuerto.values, hue=cantidad_vuelos_por_aeropuerto.index, legend=False, palette='viridis')
plt.title('Cantidad de Vuelos por Aeropuerto')
plt.xlabel('Aeropuerto')
plt.ylabel('Cantidad de Vuelos')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Obtener el top 3 de la columna 'origen_destino'
top_3_aerolineas = df_argentina_españa_comerciales['aerolinea_nombre'].value_counts().head(3)

# Mostrar los valores del top 3
top_3_aerolineas.reset_index()

In [ ]:
# Reseteamos el índice y lo convertimos a lista
lista_top_3 = top_3_aerolineas.reset_index().values.tolist()

# Extraemos solo los nombres de las aerolíneas
lista_top_3_names = top_3_aerolineas.index.tolist()

In [ ]:
lista_top_3_names

In [ ]:
# Gráfico de barra para la columna 'aerolinea_nombre'
cantidad_vuelos_por_aerolinea = df_argentina_españa_comerciales['aerolinea_nombre'].value_counts()

plt.figure(figsize=(16, 5))
sns.barplot(x=cantidad_vuelos_por_aerolinea.index, y=cantidad_vuelos_por_aerolinea.values,hue=cantidad_vuelos_por_aerolinea.index, legend=False,palette='viridis')
plt.title('Cantidad de Vuelos por Aerolínea')
plt.xlabel('Aerolínea')
plt.ylabel('Cantidad de Vuelos')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Gráfico de barra para la columna 'aeronave'
cantidad_vuelos_por_aeronave = df_argentina_españa_comerciales['aeronave'].value_counts()

plt.figure(figsize=(10, 5))
sns.barplot(x=cantidad_vuelos_por_aeronave.index, y=cantidad_vuelos_por_aeronave.values,hue=cantidad_vuelos_por_aeronave.index,legend=False, palette='viridis')
plt.title('Cantidad de Vuelos por Aeronave')
plt.xlabel('Aeronave')
plt.ylabel('Cantidad de Vuelos')
plt.xticks(rotation=45)
plt.show()

Elegimos el par EZE - LEMD

In [ ]:
aeropuerto_españa = ['LEMD']
clase_vuelo = 'Regular'
aeropuerto_base = 'EZE'

# Filtrar el DataFrame original
df_vuelos_pre_final = df_vuelos_combinado_correct_dtype[
    (df_vuelos_combinado_correct_dtype['origen_destino'].isin(aeropuerto_españa)) &
    (df_vuelos_combinado_correct_dtype['clase_de_vuelo'] == clase_vuelo) &
    (df_vuelos_combinado_correct_dtype['aeropuerto'] == aeropuerto_base)
]

num_vuelos = df_vuelos_pre_final.shape[0]
print(f"El número de vuelos en el dataset filtrado es de: {num_vuelos}")

In [ ]:
df_vuelos_pre_final.head()

In [ ]:
# Evolución de pasajeros en el tiempo

df_vuelos_pre_final.groupby('fecha_utc')['pasajeros'].sum().plot()

In [ ]:
df_final = df_vuelos_pre_final[['fecha_utc', 'pasajeros']].copy()
df_final.head()

In [ ]:
df_final.dtypes

#### 1.2.1) Analizamos la Serie Temporal

In [ ]:
# Transformamos la data a frecuencia semanal, agrupamos
df_monthly = df_final.resample('M', on='fecha_utc').sum()

# Generamos un rango mensual para el índice
monthly_index = pd.date_range(start='2021-01-01', end='2024-04-01', freq='MS')

# Nos aseguramos de que la longitud del nuevo índice matchee con la de nuestro Dataframe
if len(monthly_index) == len(df_monthly):
    df_monthly.index = monthly_index
else:
    raise ValueError(f"Length mismatch: The new index has {len(monthly_index)} elements, but the DataFrame has {len(df_monthly)} rows.")

print(df_monthly)

In [ ]:
# Descomponemos la serie temporal
decomposition = seasonal_decompose(df_monthly['pasajeros'], model='multiplicative')

# Graficamos
plt.figure(figsize=(15, 12))

plt.subplot(411)
plt.plot(df_monthly['pasajeros'], label='Original')
plt.legend(loc='upper left')

plt.subplot(412)
plt.plot(decomposition.trend, label='Tendencia')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()


**Interpretación**

* **Original**

 La gráfica muestra una tendencia general al alza, con fluctuaciones evidentes que podrían indicar un patrón estacional.

* **Tendencia**

 La línea de tendencia muestra un aumento constante en el número de pasajeros a lo largo del tiempo. Esto sugiere que la demanda general de vuelos está creciendo.

In [ ]:
# Plot the decomposed components
plt.figure(figsize=(15, 12))

plt.subplot(413)
plt.plot(decomposition.seasonal, label='Estacionalidad')
plt.legend(loc='upper left')

plt.subplot(414)
plt.plot(decomposition.resid, label='Residuos')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

**Interpretación**

* **Estacionalidad**

 La gráfica estacional muestra picos y valles periódicos dentro de cada año. Esto sugiere un patrón estacional donde el número de pasajeros aumenta y disminuye regularmente en ciertas épocas del año. Por ejemplo, los picos podrían corresponder a temporadas de vacaciones u otros períodos de alta demanda, mientras que los valles podrían coincidir con períodos de baja demanda.

* **Residuos**

 La gráfica de residuos muestra las fluctuaciones restantes que no siguen un patrón de tendencia o estacionalidad. Estas podrían deberse a factores aleatorios, eventos especiales o anomalías. Idealmente, los residuos no deberían mostrar ningún patrón, lo que indicaría que el modelo ha capturado bien las partes sistemáticas de los datos.

In [ ]:
# Establecemos 'fecha_utc' como índice
df_analysis = df_final.copy()
df_analysis.set_index('fecha_utc', inplace=True)

In [ ]:
plot_acf_pacf(df_analysis)

**Interpretación**

En los gráficos ACF y PACF de la serie temporal diferenciada:

- **ACF:** Observamos una disminución gradual de la autocorrelación a medida que aumentan los retardos, indicando que la serie tiene una dependencia temporal significativa, incluso después de la diferenciación.
- **PACF:** Los primeros retardos muestran valores significativos, pero después de unos pocos retardos, la correlación cae cerca de cero, sugiriendo que la influencia directa de los retardos se limita a los primeros lags.


In [ ]:
plot_fac(df_analysis)

**Análisis de FAC de la Serie Temporal Diferenciada**

**Propósito del Análisis:**

Buscamos entender la autocorrelación presente en la serie temporal después de la diferenciación para determinar si la serie es estacionaria e identificar patrones de dependencia temporal.

**Interpretación del Gráfico FAC**
- **Primera Observación Alta:** Indica una fuerte autocorrelación en el primer retardo, común en series diferenciadas.
- **Disminución Gradual:** La autocorrelación disminuye gradualmente, indicando una estructura de dependencia a lo largo del tiempo.
- **Valores Extremos:** Los picos en los extremos pueden indicar estacionalidad o cambios abruptos en la serie.


#### 1.2.2) Pruebas estadísticas

El objetivo principal es determinar si una serie temporal es estacionaria. Para lograrlo, vamos a utilizar tres pruebas estadísticas diferentes. La prueba de Zivot-Andrews se centrará en identificar una tendencia significativa en la media de la serie, mientras que la prueba de Dickey-Fuller Aumentada (ADF) nos ayudará a examinar la estacionariedad en la media. Por otro lado, la prueba de KPSS se enfocará en la estacionariedad en la varianza de la serie. Al realizar estas pruebas, buscamos comprender la naturaleza de la serie y determinar si son necesarias transformaciones o modelos específicos antes de proceder con su análisis y predicción.

In [ ]:
stationarity_tests(df_analysis)

**Conclusiones**

Los resultados de la prueba ADF sugieren que la serie es estacionaria, ya que su valor p es menor que 0.05 y su estadística de prueba es menor que los valores críticos. Por otro lado, la prueba KPSS también indica estacionariedad, ya que su valor p es menor que 0.05 y su estadística de prueba excede los valores críticos. En conclusión, ambas pruebas respaldan la estacionariedad de la serie temporal.

In [ ]:
test_stationarity(df_analysis)

**Conclusiones:**

Comprobamos mediante el test de Dickey-Fuller que nuestra serie es estacionaria. El valor de Test Statistic es menor que todos los valores críticos.

## 2. Análisis de importancia de Features

###2.1 Estudio PCA

In [ ]:
df_argentina_españa_comerciales.dtypes

In [ ]:
df_argentina_españa_comerciales['pax'] = df_argentina_españa_comerciales.pax.astype('int64')

In [ ]:
# Hago dummies para aquellas features las cuales pueden categorizarse
dummies_columns = ['tipo_de_movimiento', 'aerolinea_nombre', 'aeronave']

# TODO - cambiar por oneHotEncoder
df_argentina_españa_comerciales_dummies = pd.get_dummies(df_argentina_españa_comerciales, columns= dummies_columns)

In [ ]:
df_argentina_españa_comerciales_dummies.dtypes

In [ ]:
# Separar las variables numéricas
type_numeric_list = ['bool', 'datetime64']
drop_train_cols_list = ['pasajeros', 'pax']

df_train = df_argentina_españa_comerciales_dummies.select_dtypes(include= type_numeric_list)

# separo los datos de train, que abarcan los años 2021 hasta 2023, y test a usar el 2024
df_train = date_filter(df_train, 'fecha_utc', lim_inferior_entrenamiento, lim_superior_entrenamiento)
df_test = date_filter(df_train, 'fecha_utc', lim_inferior_prueba, lim_superior_prueba)

df_train_pca = df_train.copy()
df_train_pca['fecha_int64'] = df_train_pca.fecha_utc.apply(lambda x: x.strftime('%Y%m%d')).astype('int64')
df_train_pca.drop(columns= ['fecha_utc'], inplace= True)

# Escalar las variables numéricas
scaler = StandardScaler()
X_numerico_scaled = scaler.fit_transform(df_train_pca)

# Aplicar PCA
pca = PCA()
X_pca = pca.fit(X_numerico_scaled)

# Calcular la varianza explicada por cada componente principal
explained_variance_ratio = pca.explained_variance_ratio_

# Imprimir la varianza explicada acumulada
print("Varianza explicada acumulada por componente principal:")
print(explained_variance_ratio.cumsum())

In [ ]:
# Imprimir la varianza explicada acumulada
cumulative_variance = explained_variance_ratio.cumsum()
print("Varianza explicada acumulada por componente principal:")
print(cumulative_variance)

# Determinar el número de componentes necesarios para explicar el 99.99% de la varianza
num_components = np.argmax(cumulative_variance >= 0.9999) + 1
print(f"Se requieren las primeras {num_components} componentes principales para explicar el 99.99% de la varianza.")

# Transformar los datos usando solo las componentes necesarias
pca = PCA(n_components=num_components)
X_pca_reduced = pca.fit_transform(X_numerico_scaled)

# Obtener las cargas de los componentes
component_loadings = pca.components_

# Crear un DataFrame para las cargas
loadings_df = pd.DataFrame(component_loadings.T, columns=[f'PC{i+1}' for i in range(num_components)], index=df_train_pca.columns)

# Identificar las características más importantes para cada componente
important_features = {}
for i in range(num_components):
    component = f'PC{i+1}'
    sorted_loadings = loadings_df[component].abs().sort_values(ascending=False)
    important_features[component] = sorted_loadings.index.tolist()

# Encontrar características comunes en las 21 componentes principales
common_features = set(important_features['PC1'])
for i in range(2, num_components + 1):
    common_features.intersection_update(important_features[f'PC{i}'])

print("\nCaracterísticas comunes en las 21 componentes principales:")
print(common_features)

# Graficar la varianza explicada
plt.figure(figsize=(6, 5))
plt.plot(range(len(explained_variance_ratio)), explained_variance_ratio, '-o', label='Componente individual')
plt.plot(range(len(explained_variance_ratio)), cumulative_variance, '-s', label='Acumulado')
plt.axvline(x=num_components-1, color='r', linestyle='--', label=f'{num_components} componentes')
plt.ylabel('Porcentaje de Varianza Explicada')
plt.xlabel('Componentes Principales')
plt.ylim(0, 1.05)
plt.xticks(range(len(explained_variance_ratio)))
plt.legend(loc='best')
plt.show()

No es posible hacer una limpieza más exhaustiva que la realizada en el EDA con el método PCA.

## 3. Definimos Dataset final para Entrenamiento

In [ ]:
df_final_entrenamiento = df_vuelos_pre_final[['fecha_utc', 'pasajeros']].copy()

# Transformamos la data a frecuencia semanal, agrupamos
df_final_mensual_entrenamiento = df_final_entrenamiento.resample('M', on='fecha_utc').sum()

# Generamos un rango mensual para el índice
monthly_index = pd.date_range(start='2021-01-01', end='2024-04-01', freq='MS')

# Nos aseguramos de que la longitud del nuevo índice matchee con la de nuestro Dataframe
if len(monthly_index) == len(df_final_mensual_entrenamiento):
    df_final_mensual_entrenamiento.index = monthly_index
else:
    raise ValueError(f"Length mismatch: The new index has {len(monthly_index)} elements, but the DataFrame has {len(df_final_mensual_entrenamiento)} rows.")

df_final_mensual_entrenamiento.head()

## 4. Entrenamiento

## 4.1) Entrenamos con ExponentialSmoothing

#### 4.1.1) Sin estacionalidad

In [ ]:
# Obtenemos los primeros 3 años de datos
train_start_date_se = df_final_mensual_entrenamiento.index[0]  # Fecha del primer dato
train_end_date_se  = train_start_date_se + pd.DateOffset(years=3)  # Fecha después de 3 años

train_se = df_final_mensual_entrenamiento.loc[train_start_date_se:train_end_date_se]

# Entenamiento del modelo
exp_model_se = ExponentialSmoothing(train_se, trend='mul')
exp_model_fit_se = exp_model_se.fit()

# Pronóstico
forecast_steps_se = 12 # Por ejemplo, pronosticar un año después del entrenamiento
forecast_start_date_se = train_end_date_se + pd.DateOffset(months=0)  # Fecha de inicio para pronosticar
forecast_end_date_se = forecast_start_date_se + pd.DateOffset(months=forecast_steps_se - 1)  # Fecha de finalización para pronosticar
forecast_se = exp_model_fit_se.predict(start=forecast_start_date_se, end=forecast_end_date_se)

# Graficamos
plt.figure(figsize=(15, 6))
plt.plot(df_final_mensual_entrenamiento.index, df_final_mensual_entrenamiento.pasajeros, label='Datos', color='blue')
plt.plot(train_se.index, exp_model_fit_se.fittedvalues, label='Ajuste (Entrenamiento)', color='green')
plt.plot(forecast_se.index, forecast_se, label='Pronóstico', color='red')
plt.title('Modelo de Suavizado Exponencial')
plt.xlabel('Fecha')
plt.ylabel('Número de pasajeros')
plt.legend()
plt.show()

In [ ]:
train_forecast_se = exp_model_fit_se.predict(start=0, end=36)
test_forecast_se = exp_model_fit_se.predict(start=36, end=40)

test_se = df_final_mensual_entrenamiento[-5:]

##### Analizamos métricas de performance

In [ ]:
print('RMSE train: ', np.sqrt(mean_squared_error(train_se, train_forecast_se)))
print('RMSE test: ', np.sqrt(mean_squared_error(test_se, test_forecast_se)))
print('MAE train: ', mean_absolute_error(train_se, train_forecast_se))
print('MAE test: ', mean_absolute_error(test_se, test_forecast_se))

##### Analizamos residuos

In [ ]:
plt.figure(figsize=(35,10))
plt.plot(exp_model_fit_se.resid)

Observaciones

Se observa un comportamiento aleatorio en los residuos, lo que implica que no hay patrones presentes. Esto sugiere que el modelo entrenado ha capturado adecuadamente las tendencias y estacionalidades relevantes de los datos.

### 4.1.2) Con estacionalidad

In [ ]:
# Obtenemos los primeros 3 años de datos
train_start_date_ce = df_final_mensual_entrenamiento.index[0]  # Fecha del primer dato
train_end_date_ce = train_start_date_ce + pd.DateOffset(years=3)  # Fecha después de 3 años

train_ce = df_final_mensual_entrenamiento.loc[train_start_date_ce:train_end_date_ce]

# Entenamiento del modelo
exp_model_ce = ExponentialSmoothing(train_ce, seasonal='add', trend='mul', seasonal_periods=12)
exp_model_fit_ce = exp_model_ce.fit()

# Pronóstico
forecast_steps_ce = 12 # Por ejemplo, pronosticar un año después del entrenamiento
forecast_start_date_ce = train_end_date_ce + pd.DateOffset(months=0)  # Fecha de inicio para pronosticar
forecast_end_date_ce = forecast_start_date_ce + pd.DateOffset(months=forecast_steps_ce - 1)  # Fecha de finalización para pronosticar
forecast_ce = exp_model_fit_ce.predict(start=forecast_start_date_ce, end=forecast_end_date_ce)

# Graficamos
plt.figure(figsize=(35, 10))
plt.plot(df_final_mensual_entrenamiento.index, df_final_mensual_entrenamiento.pasajeros, label='Datos', color='blue')
plt.plot(train_ce.index, exp_model_fit_ce.fittedvalues, label='Ajuste (Entrenamiento)', color='green')
plt.plot(forecast_ce.index, forecast_ce, label='Pronóstico', color='red')
plt.title('Modelo de Suavizado Exponencial')
plt.xlabel('Fecha')
plt.ylabel('Número de pasajeros')
plt.legend()
plt.show()

In [ ]:
train_forecast_ce = exp_model_fit_ce.predict(start=0, end=36)
test_forecast_ce = exp_model_fit_ce.predict(start=36, end=40)

In [ ]:
test_ce = df_final_mensual_entrenamiento[-5:]

##### Analizamos métricas de performance

In [ ]:
print('RMSE train: ', np.sqrt(mean_squared_error(train_ce, train_forecast_ce)))
print('RMSE test: ', np.sqrt(mean_squared_error(test_ce, test_forecast_ce)))
print('MAE train: ', mean_absolute_error(train_ce, train_forecast_ce))
print('MAE test: ', mean_absolute_error(test_ce, test_forecast_ce))

##### Analizamos residuos

In [ ]:
plt.figure(figsize=(35,10))
plt.plot(exp_model_fit_ce.resid)

NameError: name 'exp_model_fit_ce' is not defined

<Figure size 3500x1000 with 0 Axes>

Observaciones

Se observa un comportamiento aleatorio en los residuos, lo que implica que no hay patrones presentes. Esto sugiere que el modelo entrenado ha capturado adecuadamente las tendencias y estacionalidades relevantes de los datos.

### 4.1.3) Hiperparámetros para ExponentialSmoothing

In [ ]:
alphas = betas = gammas = np.arange(0.10, 1, 0.20)
abg = list(itertools.product(alphas, betas, gammas))

In [ ]:
best_alpha, best_beta, best_gamma, best_mae = tes_optimizer(train_ce, test_ce, abg)

In [ ]:
# Entrenar el modelo de suavizado exponencial triplano con hiperparámetros
final_tes_model = ExponentialSmoothing(
    train_ce,
    trend="add",
    seasonal="add",
    seasonal_periods=12
).fit(
    smoothing_level=best_alpha,
    smoothing_trend=best_beta,
    smoothing_seasonal=best_gamma
)

# Obtener la fecha de inicio y fin del pronóstico
forecast_start_date_hp = train_ce.index[-1] + pd.DateOffset(months=0)  # Comienza el mismo mes que en train_ce
forecast_end_date_hp = forecast_start_date_hp + pd.DateOffset(months=11)  # Cubre un período de 12 meses

# Generar el pronóstico
y_pred_hp = final_tes_model.predict(start=forecast_start_date_hp, end=forecast_end_date_hp)

# Visualizar los datos, ajuste del modelo sin hiperparámetros, pronóstico del modelo sin hiperparámetros y pronóstico del modelo con hiperparámetros
plt.figure(figsize=(15, 6))
plt.plot(df_final_mensual_entrenamiento.index, df_final_mensual_entrenamiento['pasajeros'], label='Datos', color='blue')
plt.plot(train_ce.index, exp_model_fit_ce.fittedvalues, label='Ajuste (Modelo sin hiperparámetros)', color='green', linestyle=":")
plt.plot(train_ce.index, final_tes_model.fittedvalues, label='Ajuste (Modelo con hiperparámetros)', color='orange')
plt.plot(forecast_ce.index, forecast_ce, label='Pronóstico (Modelo sin hiperparámetros)', color='red', linestyle=":")
plt.plot(y_pred_hp.index, y_pred_hp, label='Pronóstico (Modelo con hiperparámetros)', color='purple')
plt.title('Comparación de Modelos de Suavizado Exponencial')
plt.xlabel('Fecha')
plt.ylabel('Número de pasajeros')
plt.legend()
plt.show()

#### Analizamos métricas de performance

In [ ]:
train_forecast_hp = final_tes_model.predict(start=0, end=36)
test_forecast_hp = final_tes_model.predict(start=36, end=40)

test_hp = df_final_mensual_entrenamiento[-5:]

print('RMSE train: ', np.sqrt(mean_squared_error(train_ce, train_forecast_hp)))
print('RMSE test: ', np.sqrt(mean_squared_error(test_hp, test_forecast_hp)))
print('MAE train: ', mean_absolute_error(train_ce, train_forecast_hp))
print('MAE test: ', mean_absolute_error(test_hp, test_forecast_hp))

#### Analizamos residuos

In [ ]:
plt.figure(figsize=(35,10))
plt.plot(final_tes_model.resid)

Observaciones

Se observa un comportamiento aleatorio en los residuos, lo que implica que no hay patrones presentes. Esto sugiere que el modelo entrenado ha capturado adecuadamente las tendencias y estacionalidades relevantes de los datos.



## 4.2) Entrenamos con SARIMAX

En este modelo podemos sumar las dummies al análisis.

### 4.2.1) Buscamos Hiperparámetros para SARIMAX

In [ ]:
ts = pd.Series(df_final_mensual_entrenamiento['pasajeros'])
p = d = q = range(0, 3)
P = D = Q = range(0, 2)
s = [12]  # Para datos mensuales

mejor_modelo = busqueda_sarimax(ts, p, d, q, P, D, Q, s)
print("Mejores parámetros del modelo SARIMAX:", mejor_modelo)

### 4.2.2) Con hiperparámetros, estacionalidad y sin dummies

In [ ]:
# Guardar parámetros obtenidos en una variable
mejores_params = mejor_modelo['params']

# Dividir los datos: los primeros 36 meses para entrenamiento, el resto para prueba
train_sm = df_final_mensual_entrenamiento.iloc[:36]
test_sm = df_final_mensual_entrenamiento.iloc[36:48]

# Ajustar el modelo SARIMAX en el conjunto de entrenamiento
model_sm = SARIMAX(train_sm['pasajeros'], order=mejores_params[0], seasonal_order=mejores_params[1])
results_sm = model_sm.fit()

# Hacer predicciones en el conjunto de prueba
predictions_sm = results_sm.get_forecast(steps=len(test_sm))
predicted_mean_sm = predictions_sm.predicted_mean
conf_int_sm = predictions_sm.conf_int()

# Hacer predicciones en el conjunto de entrenamiento
train_predictions_sm = results_sm.predict(start=train_sm.index[0], end=train_sm.index[-1])

train_predictions_sm_to_plot = results_sm.predict(start=train_sm.index[0], end=test_sm.index[0]) # se crea para que grafique completo el entrenamiento

# Graficar los datos originales, las predicciones y el intervalo de confianza
plt.figure(figsize=(15, 6))
plt.plot(df_final_mensual_entrenamiento.index, df_final_mensual_entrenamiento['pasajeros'], label='Datos reales', color='blue')
plt.plot(test_sm.index, test_sm['pasajeros'], label='Datos de prueba', color='red')
plt.plot(predicted_mean_sm.index, predicted_mean_sm, label='Predicciones (Prueba)', linestyle='--', color='purple')
plt.plot(train_predictions_sm_to_plot.index, train_predictions_sm_to_plot, label='Predicciones (Entrenamiento)', linestyle='--', color='orange')
plt.fill_between(predicted_mean_sm.index, conf_int_sm.iloc[:, 0], conf_int_sm.iloc[:, 1], color='k', alpha=0.1)
plt.legend()
plt.title('Modelo SARIMAX')
plt.xlabel('Fecha')
plt.ylabel('Número de pasajeros')
plt.show()

#### Analizamos métricas de performance

In [ ]:
# Imprimir el resumen del modelo ajustado
print(results_sm.summary())

In [ ]:
mae_test = mean_absolute_error(test_sm['pasajeros'], predicted_mean_sm)
rmse_test = np.sqrt(mean_squared_error(test_sm['pasajeros'], predicted_mean_sm))
mae_train = mean_absolute_error(train_sm['pasajeros'], train_predictions_sm)
rmse_train = np.sqrt(mean_squared_error(train_sm['pasajeros'], train_predictions_sm))

print(f'RMSE entrenamiento: {rmse_train}')
print(f'RMSE prueba: {rmse_test}')
print(f'Error Medio Absoluto prueba (MAE): {mae_test}')
print(f'Error Medio Absoluto entrenamiento (MAE): {mae_train}')

#### Analizamos residuos

In [ ]:
plt.figure(figsize=(35,10))
plt.plot(results_sm.resid)

Observaciones

Se observa un comportamiento random en los residuos, lo que implica que no hay patrones presentes. Esto sugiere que el modelo entrenado ha capturado las tendencias y estacionalidades relevantes de los datos.

### 4.2.3) Buscamos hiperparámetros con gridsearchCV en un pipeline

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Dividir los datos: los primeros 30 meses para entrenamiento, el resto para prueba
train = df_final_mensual_entrenamiento.iloc[:30]
test = df_final_mensual_entrenamiento.iloc[30:48]

# Función para ajustar SARIMAX
def sarimax_fit(X, y=None, order=(1, 0, 0), seasonal_order=(1, 1, 1, 12)):
    model = SARIMAX(X, order=order, seasonal_order=seasonal_order)
    results = model.fit(disp=False)
    return results

# Función para predecir usando un modelo ajustado de SARIMAX
def sarimax_predict(results, steps):
    forecast = results.get_forecast(steps=steps)
    return forecast.predicted_mean

# Función para hacer la predicción en el pipeline
def predict_with_sarimax(X, order=(1, 0, 0), seasonal_order=(1, 1, 1, 12)):
    model = sarimax_fit(X, order=order, seasonal_order=seasonal_order)
    return sarimax_predict(model, steps=len(X))

# Crear transformadores con FunctionTransformer
fit_transformer = FunctionTransformer(sarimax_fit, kw_args={'order': (1, 0, 0), 'seasonal_order': (1, 1, 1, 12)}, validate=False)
predict_transformer = FunctionTransformer(predict_with_sarimax, kw_args={'order': (1, 0, 0), 'seasonal_order': (1, 1, 1, 12)}, validate=False)

# Definir el pipeline
pipeline = Pipeline([
    ('fit', fit_transformer),
    ('predict', predict_transformer)
])

# Definir el grid de parámetros
param_grid = {
    'fit__kw_args': [{'order': (1, 0, 0), 'seasonal_order': (1, 1, 1, 12)},
                     {'order': (1, 1, 0), 'seasonal_order': (1, 0, 1, 12)},
                     {'order': (0, 1, 1), 'seasonal_order': (0, 1, 1, 12)}]
}

# Definir el cross-validator
tscv = TimeSeriesSplit(n_splits=3)

# Definir el GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=tscv, scoring='neg_mean_squared_error')

# Ajustar el GridSearchCV
grid_search.fit(train['pasajeros'].values.reshape(-1, 1))

# Mejor modelo encontrado
best_params = grid_search.best_params_
order = best_params['fit__kw_args']['order']
seasonal_order = best_params['fit__kw_args']['seasonal_order']

# Ajustar el mejor modelo en los datos de entrenamiento completos
best_model_results = sarimax_fit(train['pasajeros'], order=order, seasonal_order=seasonal_order)

# Hacer predicciones en el conjunto de prueba
predicted_mean = sarimax_predict(best_model_results, steps=len(test))

# Visualizar los resultados
plt.figure(figsize=(10, 6))
plt.plot(train.index, train['pasajeros'], label='Entrenamiento')
plt.plot(test.index, test['pasajeros'], label='Prueba', color='orange')
plt.plot(test.index, predicted_mean, label='Predicciones', color='green')
plt.legend()
plt.title('Predicción de pasajeros usando SARIMAX con GridSearchCV')
plt.xlabel('Fecha')
plt.ylabel('Número de pasajeros')
plt.show()

mae = mean_absolute_error(test['pasajeros'], predicted_mean)
print('mae', mae)

rmse = np.sqrt(mean_squared_error(test['pasajeros'], predicted_mean))
print('rmse', rmse)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros:", best_params)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np

class SarimaxFit(BaseEstimator, TransformerMixin):
    def __init__(self, order=(1, 0, 0), seasonal_order=(1, 1, 1, 12)):
        self.order = order
        self.seasonal_order = seasonal_order
        self.results_ = None

    def fit(self, X, y=None):
        self.model_ = SARIMAX(X, order=self.order, seasonal_order=self.seasonal_order)
        self.results_ = self.model_.fit(disp=False)
        return self

    def transform(self, X, y=None):
        return self

    def get_results(self):
        return self.results_

class SarimaxPredict(BaseEstimator, TransformerMixin):
    def __init__(self, steps=None):
        self.steps = steps
        self.forecast_ = None

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.steps is None:
            self.steps = len(X)
        self.forecast_ = self.results_.get_forecast(steps=self.steps)
        return self.forecast_.predicted_mean


# Pipeline que ajusta y predice usando SARIMAX
sarimax_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('sarimax_fit', SarimaxFit()),
    ('sarimax_predict', SarimaxPredict())
])

param_grid = {
    'sarimax_fit__order': [(1, 0, 0), (1, 1, 0), (2, 1, 0)],
    'sarimax_fit__seasonal_order': [(1, 1, 1, 12), (1, 0, 1, 12)],
    'sarimax_predict__steps': [10, 20, 30]
}

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Configuración de la validación cruzada para series temporales
tscv = TimeSeriesSplit(n_splits=5)

# Configuración de GridSearchCV
grid_search = GridSearchCV(sarimax_pipeline, param_grid, cv=tscv, scoring='neg_mean_squared_error')

# Datos de ejemplo (reemplaza con tus datos reales)
X = np.random.rand(100)  # Serie temporal con 100 puntos

X = y.reshape(-1, 1)

# Ajustar el grid search
grid_search.fit(X)

# Mejor configuración de parámetros y mejor puntaje
print("Mejores parámetros encontrados: ", grid_search.best_params_)
print("Mejor puntaje (neg MSE): ", grid_search.best_score_)
print("Mejor puntaje (MSE): ", -grid_search.best_score_)


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

train = df_monthly.iloc[:30]
test = df_monthly.iloc[30:48]

# Función de ajuste
def sarimax_fit(X, y=None, order=(1, 0, 0), seasonal_order=(1, 1, 1, 12)):
    model = SARIMAX(X, order=order, seasonal_order=seasonal_order)
    results = model.fit(disp=False)
    return results

# Función de prediccion
def sarimax_predict(results, steps):
    forecast = results.get_forecast(steps=steps)
    return forecast.predicted_mean

# Función de prediccion en el pipeline
def predict_with_sarimax(X, order=(1, 0, 0), seasonal_order=(1, 1, 1, 12)):
    model = sarimax_fit(X, order=order, seasonal_order=seasonal_order)
    return sarimax_predict(model, steps=len(X))

fit_transformer = FunctionTransformer(sarimax_fit, kw_args={'order': (1, 0, 0), 'seasonal_order': (1, 1, 1, 12)}, validate=False)
predict_transformer = FunctionTransformer(predict_with_sarimax, kw_args={'order': (1, 0, 0), 'seasonal_order': (1, 1, 1, 12)}, validate=False)

# Definir el pipeline
pipeline = Pipeline([
    ('fit', fit_transformer),
    ('predict', predict_transformer)
])


order_1 = (1, 0, 0)
order_2 = (1, 1, 0)
order_3 = (0, 1, 1)

# Definir el grid de parámetros
param_grid = {
    'fit__kw_args': [{'order': order_1, 'seasonal_order': (1, 1, 1, 12)},
                     {'order': order_2, 'seasonal_order': (1, 0, 1, 12)},
                     {'order': order_3, 'seasonal_order': (0, 1, 1, 12)}]
}

tscv = TimeSeriesSplit(n_splits=3)

# Se define un GridSearchCV

scoring_gs_cv = 'neg_mean_squared_error'
grid_search = GridSearchCV(pipeline, param_grid, cv=tscv, scoring= scoring_gs_cv)

# Ajustar el GridSearchCV
grid_search.fit(train['pasajeros'].values.reshape(-1, 1))

# Mejor modelo encontrado
best_params = grid_search.best_params_
order = best_params['fit__kw_args']['order']
seasonal_order = best_params['fit__kw_args']['seasonal_order']

# Ajustar el mejor modelo en los datos de entrenamiento completos
best_model_results = sarimax_fit(train['pasajeros'], order=order, seasonal_order=seasonal_order)

# Se hace el predict
predicted_mean = sarimax_predict(best_model_results, steps=len(test))

# Indicadores de error
mae = mean_absolute_error(test['pasajeros'], predicted_mean)
print('mae', mae)

rmse = np.sqrt(mean_squared_error(test['pasajeros'], predicted_mean))
print('rmse', rmse)

# Mejores parámetros encontrados
print("Mejores parametros:", best_params)

# Best model
print("Mejores modelo:", grid_search.best_estimator_)


# Visualizar los resultados
plt.figure(figsize=(10, 6))
plt.plot(train.index, train['pasajeros'], label='Entrenamiento')
plt.plot(test.index, test['pasajeros'], label='Prueba', color='orange')
plt.plot(test.index, predicted_mean, label='Predicciones', color='green')
plt.legend()
plt.title('Predicción de pasajeros usando SARIMAX con GridSearchCV')
plt.xlabel('Fecha')
plt.ylabel('Número de pasajeros')
plt.show()


## 5. Análisis pronóstico de pasajeros y precio

**Clasificación de precios de boletos:**

Se determina la cantidad de pasajeros por año para cada mes. Según la cantidad máxima y mínima de ese año, definimos umbrales que nos permitan definir si la demanda es baja, media o alta. Esa demanda la extrapolamos a un precio bajo, medio o alto.

## 6. Conclusiones

Nuestro objetivo es realizar pronósticos de vuelos desde Argentina a Madrid, con la finalidad de predecir cuándo sería más barato viajar en función de la cantidad de pasajeros que viajarían. En este contexto, las métricas de error son cruciales para asegurar la precisión y confiabilidad de nuestras predicciones.

Después de evaluar las métricas de performance de los modelos entrenados, hemos decidido avanzar con la utilización del algoritmo Exponential Smoothing debido a su destacada performance. A continuación, se presentan las métricas obtenidas para este modelo:

* RMSE (Root Mean Squared Error) en el conjunto de entrenamiento: 7545.16
* RMSE en el conjunto de prueba: 2755.03
* MAE (Mean Absolute Error) en el conjunto de entrenamiento: 5933.33
* MAE en el conjunto de prueba: 2588.36

Las razones principales para elegir el modelo de Exponential Smoothing son las siguientes:

Precisión en el conjunto de prueba: El modelo de Exponential Smoothing ha demostrado tener los valores de RMSE y MAE más bajos en el conjunto de prueba comparado con los otros modelos entrenados. Esto indica que el modelo tiene una mejor capacidad para generalizar y predecir datos no vistos previamente.

Consistencia en las métricas: Los valores de RMSE y MAE en el conjunto de entrenamiento también son relativamente bajos, lo que sugiere que el modelo está bien ajustado a los datos de entrenamiento sin sobreajustarse.

Dado nuestro objetivo de predecir la demanda de vuelos para identificar períodos de menor costo de viaje, la precisión en las predicciones es fundamental. Las métricas de error, especialmente en el conjunto de prueba, son indicativas de la capacidad del modelo para hacer predicciones precisas. Un error más bajo en el conjunto de prueba sugiere que el modelo de Exponential Smoothing es más adecuado para realizar pronósticos fiables sobre la cantidad de pasajeros y, por ende, sobre los períodos más económicos para viajar.